# 세팅

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 50.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import konlpy
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
data = pd.read_excel('/content/식품_train.xlsx')
data

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,식품,46175,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,컵밥|죽,46175,1
1,2,s,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,식품,46175,a,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46175,2
2,3,c,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,식품,40543,q,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,해마로|볶음밥,40543,1
3,4,s,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,식품,40543,a,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40543,2
4,5,c,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,식품,15426,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15426,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434307,434308,s,지금 주문주신다면 내일 출고되어 토요일에 받아보실 수 있으세요. 참고 하시어 이용에...,식품,83035,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,토요일,NaN,83035,2
434308,434309,c,지금 주문했는데요 내일 꼭 받아야 합니다 가능한지요?,식품,109688,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,1
434309,434310,s,주문건은 오늘 발송됩니다. 송장번호 조회는 저녁 6시30분 이후부터 가능하세요.,식품,109688,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,2
434310,434311,c,지금 사면 수요일에 배송받을 수 있나요?,식품,135868,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,수요일,NaN,135868,1


# 제품 키워드 토픽 모델링

In [ ]:
df = data[(data['인텐트 (대)']=='제품')]
text = df[['발화문']]
text.head(5)

,발화문
2988,혹시 일리네스 프레소 캡슐 클래시 코랑룽고 차이가요룽고는 물양 많이 뽑아도 클래시코...
2989,룽고 제품이 클래시코 제품보다 추출량이 더 많습니다.
2990,꿀밤고구마랑 그냥 밤고구마랑 뭐가 다른 가요?
2991,세척 꿀밤고구마 2018년 햇꿀고구마부터 출고될 예정입니다.
2992,숄더랙을 주문하고 싶은데 냉장과 냉동의 차이가 클까요?


## 텍스트 전처리

In [ ]:
# konlpy의 Okt 객체를 생성
okt = Okt()

# 'headline_text' 열의 각 행에 대해 한국어 토큰화 수행
def tokenize_korean_text(row):
    return okt.nouns(row['발화문'])

In [ ]:
text['발화문'] = text.apply(tokenize_korean_text, axis=1)

<ipython-input-7-2bd7be021410>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['발화문'] = text.apply(tokenize_korean_text, axis=1)


In [ ]:
#불용어 리스트
import chardet

# 파일의 인코딩을 감지합니다.
file_path = '/content/stopwords-ko.txt'
with open(file_path, 'rb') as file:
    result = chardet.detect(file.read())

# 감지된 인코딩을 사용하여 파일을 다시 엽니다.
correct_encoding = result['encoding']
with open(file_path, 'r', encoding=correct_encoding) as file:
    stopwords = [line.strip() for line in file.readlines()]

print(stopwords)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

In [ ]:
# 추가할 새로운 불용어 리스트
new_stop_words = ["정도", "건가", "가요", "주시", "주신", "대해", "때문", "혹시", "그냥", "해주시", "대로", "드릴", "가지"]

# 새로운 불용어를 기존 불용어 리스트에 추가
stopwords.extend(new_stop_words)

# 불용어 리스트 출력
print(stopwords)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

In [ ]:
# 텍스트에서 한국어 불용어를 제거하는 함수
# 'headline_text' 열에 있는 각 행에 대해 불용어를 제거
def remove_stopwords(tokens):
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return tokens

# 'headline_text' 열에 있는 각 행에 대해 불용어를 제거한 결과로 열을 업데이트
text['발화문'] = text['발화문'].apply(remove_stopwords)

<ipython-input-10-dc2814092b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['발화문'] = text['발화문'].apply(remove_stopwords)


In [ ]:
text

,발화문
2988,"[일리, 네스, 프레, 소, 캡슐, 클래시, 코랑룽, 룽, 물, 클래시, 코, 개, 맛]"
2989,"[룽, 제품, 클래시, 코, 제품, 추출량, 더]"
2990,"[꿀밤, 고구마, 밤, 고구마, 뭐]"
2991,"[세척, 꿀밤, 고구마, 햇꿀, 고구마, 출고, 예정]"
2992,"[숄더랙, 주문, 냉장, 냉동, 차이, 요]"
...,...
431611,"[해당, 내용, 담당, 부서, 전달]"
431612,"[맛, 수도, 이건, 맛, 독소, 목구멍, 복통, 거, 독소, 감자]"
431613,"[반품, 택배, 기사, 방문, 시오, 반품, 후, 확인, 조치]"
431614,"[중, 개, 문제, 통화, 기억]"


In [ ]:
tokenized_doc = text['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc[:5])

2988    [일리, 네스, 프레, 캡슐, 클래시, 코랑룽, 클래시]
2989                 [제품, 클래시, 제품, 추출량]
2990                     [꿀밤, 고구마, 고구마]
2991     [세척, 꿀밤, 고구마, 햇꿀, 고구마, 출고, 예정]
2992              [숄더랙, 주문, 냉장, 냉동, 차이]
Name: 발화문, dtype: object


## 정수 인코딩

In [ ]:
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(4, 1), (6, 2), (7, 1)]


In [ ]:
print(dictionary[45])

김치


In [ ]:
len(dictionary) #총 학습된 단어의 개수

22627

## LDA 모델 훈련시키기

In [ ]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.116*"고객" + 0.104*"선택" + 0.070*"사이즈" + 0.052*"생각"')
(1, '0.437*"가격" + 0.072*"추가" + 0.057*"제조" + 0.035*"안심"')
(2, '0.200*"판매" + 0.116*"현재" + 0.063*"옵션" + 0.062*"상품"')
(3, '0.091*"말씀" + 0.087*"행사" + 0.085*"홍삼" + 0.049*"자연"')
(4, '0.145*"냉동" + 0.102*"냉장" + 0.066*"상태" + 0.055*"일리"')
(5, '0.155*"성분" + 0.115*"함유" + 0.062*"재료" + 0.056*"생산"')
(6, '0.225*"상품" + 0.174*"문의" + 0.119*"참고" + 0.040*"표기"')
(7, '0.278*"사용" + 0.105*"머신" + 0.080*"가루" + 0.048*"뭔가"')
(8, '0.165*"차이" + 0.061*"프로" + 0.060*"할인" + 0.047*"양념"')
(9, '0.163*"섭취" + 0.111*"캡슐" + 0.109*"제품" + 0.071*"경우"')
(10, '0.146*"함량" + 0.095*"고구마" + 0.072*"설탕" + 0.057*"단백질"')
(11, '0.114*"호환" + 0.076*"네스프레소" + 0.073*"발송" + 0.068*"출고"')
(12, '0.227*"주문" + 0.140*"확인" + 0.068*"배송" + 0.058*"상품"')
(13, '0.150*"하루" + 0.081*"크기" + 0.074*"오늘" + 0.061*"보통"')
(14, '0.140*"포함" + 0.076*"소스" + 0.073*"보시" + 0.054*"용량"')
(15, '0.224*"보관" + 0.114*"포장" + 0.101*"비타민" + 0.064*"칼로리"')
(16, '0.389*"제품" + 0.123*"유통" + 0.116*"기한" + 0.034*"유산균"')
(17, '0.098*"첨가

In [ ]:
# 파라미터 설정
num_topics = 7    # 토픽 수
alpha = 0.9       # 문서-토픽 밀도
eta = 0.6         # 단어-토픽 밀도
chunksize = 200   # 학습청크 당 문서 수
passes = 20       # 전체 말뭉치 학습 횟수
iterations = 300  # 토픽비율 추정에서 루프 최대 반복 횟수
random_state = 42 # 결과의 재현가능성을 위해 설정

ldamodel = LdaModel(
    num_topics=num_topics,
    alpha = alpha,
    eta = eta,
    corpus=corpus,
    id2word=dictionary,
    chunksize=chunksize,
    iterations=iterations,
    passes=passes
)

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
print(ldamodel.print_topics())

In [ ]:
# Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_doc, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
# Perplexity score: 신뢰성 평가 기준, 값이 클수록 신뢰성 높음
perplexity_lda = ldamodel.log_perplexity(corpus)
print('Perplexity Score: ', perplexity_lda)

## 시각화

In [ ]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 128.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 

In [ ]:
!pip install pandas==1.5.3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.1
    Uninstalling pandas-2.1.1:
      Successfully uninstalled pandas-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary) #mds='tsne'
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 배송 인텐트 토픽 모델링

In [ ]:
df2 = data[(data['인텐트 (대)']=='배송')]
text2 = df2[['발화문']]
text2.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...
1,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.
2,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...
3,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.
4,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...


In [ ]:
# konlpy의 Okt 객체를 생성
okt = Okt()

# 'headline_text' 열의 각 행에 대해 한국어 토큰화 수행
def tokenize_korean_text(row):
    return okt.nouns(row['발화문'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
text2['발화문'] = text2.apply(tokenize_korean_text, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-27-bd4444ddd4c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text2['발화문'] = text2.apply(tokenize_korean_text, axis=1)


In [ ]:
text2['발화문'] = text2['발화문'].apply(remove_stopwords)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-28-742f9ae69bb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text2['발화문'] = text2['발화문'].apply(remove_stopwords)


In [ ]:
tokenized_doc2 = text2['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc2[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                [컵밥, 사면, 무료, 배송, 죽도, 추가, 주문, 추가, 배송비]
1                                 [박스, 합포, 수도, 답변, 양해]
2    [해마, 기준, 볶음밥, 기준, 해마, 케이준, 볶음밥, 구매, 배송비, 내야, 하나요]
3                   [제품, 최소, 수량, 부과, 이상은, 시스템, 자동, 부과]
4                           [마트, 원데이, 무료, 배송, 결제, 배송비]
Name: 발화문, dtype: object


In [ ]:
from gensim import corpora
dictionary2 = corpora.Dictionary(tokenized_doc2)
corpus2 = [dictionary2.doc2bow(text2) for text2 in tokenized_doc2]
print(corpus2[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [ ]:
import gensim
NUM_TOPICS = 10 # 10개의 토픽, k=10
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus2, num_topics = NUM_TOPICS, id2word=dictionary2, passes=15)
topics2 = ldamodel2.print_topics(num_words=4)
for topic in topics2:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.091*"박스" + 0.064*"택배" + 0.054*"제주도" + 0.040*"추가"')
(1, '0.214*"오늘" + 0.150*"주문" + 0.112*"발송" + 0.106*"내일"')
(2, '0.083*"송장" + 0.078*"주문" + 0.069*"주말" + 0.066*"번호"')
(3, '0.626*"배송" + 0.058*"묶음" + 0.057*"주문" + 0.037*"목요일"')
(4, '0.175*"발송" + 0.083*"월요일" + 0.061*"출고" + 0.053*"당일"')
(5, '0.158*"출고" + 0.070*"상품" + 0.043*"현재" + 0.041*"확인"')
(6, '0.058*"방금" + 0.042*"주문" + 0.040*"냉동" + 0.039*"배달"')
(7, '0.168*"택배" + 0.055*"변경" + 0.052*"제품" + 0.052*"요청"')
(8, '0.178*"배송" + 0.129*"확인" + 0.073*"완료" + 0.060*"주문"')
(9, '0.132*"상품" + 0.123*"수령" + 0.118*"구매" + 0.105*"금요일"')


In [ ]:
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim_models.prepare(ldamodel2, corpus2, dictionary2) #mds='tsne'
pyLDAvis.display(vis2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 행사 키워드 토픽 모델링

In [ ]:
df3 = data[(data['인텐트 (대)']=='행사')]
text3 = df3[['발화문']]
text3.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
43469,두 개 구매하면 사은품 보내주시는 거 아닌가요?
43470,문의주 신상품은 옵션 수량 2개 구매 시 사은품 증정되는 상품으로 추가 구성은 해당...
43471,파채소스는 갈비 본살만 주는 건가요?
43472,갈비 본살 주문시에만 증정됩니다.
43473,저도 7일에 주문했거든요?


In [ ]:
text3['발화문'] = text3.apply(tokenize_korean_text, axis=1)
text3['발화문'] = text3['발화문'].apply(remove_stopwords)
tokenized_doc3 = text3['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc3[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-36-3169df0125d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text3['발화문'] = text3.apply(tokenize_korean_text, axis=1)


43469                                             [구매, 사은]
43470    [의주, 상품, 옵션, 수량, 구매, 사은, 증정, 상품, 추가, 구성은, 해당, ...
43471                                         [파채, 소스, 갈비]
43472                                             [갈비, 증정]
43473                                                 [주문]
Name: 발화문, dtype: object


<ipython-input-36-3169df0125d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text3['발화문'] = text3['발화문'].apply(remove_stopwords)


In [ ]:
dictionary3 = corpora.Dictionary(tokenized_doc3)
corpus3 = [dictionary3.doc2bow(text3) for text3 in tokenized_doc3]
print(corpus3[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]


In [ ]:
NUM_TOPICS = 10 # 10개의 토픽, k=10
ldamodel3 = gensim.models.ldamodel.LdaModel(corpus3, num_topics = NUM_TOPICS, id2word=dictionary3, passes=15)
topics3 = ldamodel3.print_topics(num_words=4)
for topic in topics3:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.179*"주문" + 0.062*"박스" + 0.056*"추가" + 0.044*"사은"')
(1, '0.142*"발송" + 0.068*"사은" + 0.056*"양해" + 0.038*"상품"')
(2, '0.083*"확인" + 0.075*"오늘" + 0.059*"주문" + 0.044*"사은"')
(3, '0.094*"요청" + 0.092*"출고" + 0.055*"커피" + 0.053*"배송"')
(4, '0.179*"사은" + 0.069*"주문" + 0.069*"지금" + 0.035*"배송"')
(5, '0.071*"쉐이크" + 0.070*"사은" + 0.030*"대신" + 0.025*"스푼"')
(6, '0.204*"샘플" + 0.042*"배즙" + 0.039*"원두" + 0.037*"주문"')
(7, '0.150*"행사" + 0.131*"이벤트" + 0.088*"진행" + 0.060*"현재"')
(8, '0.092*"제품" + 0.073*"사은" + 0.070*"옵션" + 0.058*"주문"')
(9, '0.230*"구매" + 0.147*"증정" + 0.077*"사은" + 0.076*"상품"')


In [ ]:
pyLDAvis.enable_notebook()
vis3 = pyLDAvis.gensim_models.prepare(ldamodel3, corpus3, dictionary3)
pyLDAvis.display(vis3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 포장 인텐트 토픽 모델링

In [ ]:
df4 = data[(data['인텐트 (대)']=='포장')]
text4 = df4[['발화문']]
text4.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
15886,1번이 세일 중이어서 3번 말고 1번으로 2개 구매하려고 하는데요 이 두 개가 한 ...
15887,1번 1P 짜리 제품은 1P 전용 페이퍼백이 동봉되어 배송되는 제품이랍니다.
15888,포장용기가 뭔가요?
15889,네.본 제품에 사용된 포장용기의 재질은 폴리에틸렌과 폴리프로필렌 소재로 가열과정에서...
15890,택배용 박스 말고 겉박스 따로 없나요?


In [ ]:
text4['발화문'] = text4.apply(tokenize_korean_text, axis=1)
text4['발화문'] = text4['발화문'].apply(remove_stopwords)
tokenized_doc4 = text4['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc4[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-41-407f3bb7ebd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text4['발화문'] = text4.apply(tokenize_korean_text, axis=1)


15886                                             [구매, 쇼핑]
15887                       [제품, 전용, 페이퍼백, 동봉, 배송, 제품, 랍니]
15888                                         [포장, 용기, 뭔가]
15889    [제품, 사용, 포장, 용기, 재질, 폴리에틸렌, 폴리프로필렌, 소재, 가열, 과정...
15890                                         [택배, 박스, 박스]
Name: 발화문, dtype: object


<ipython-input-41-407f3bb7ebd0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text4['발화문'] = text4['발화문'].apply(remove_stopwords)


In [ ]:
dictionary4 = corpora.Dictionary(tokenized_doc4)
corpus4 = [dictionary4.doc2bow(text4) for text4 in tokenized_doc4]
print(corpus4[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_TOPICS = 10 # 10개의 토픽, k=10
ldamodel4 = gensim.models.ldamodel.LdaModel(corpus4, num_topics = NUM_TOPICS, id2word=dictionary4, passes=15)
topics4 = ldamodel4.print_topics(num_words=4)
for topic in topics4:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.130*"포장" + 0.103*"배송" + 0.061*"진공" + 0.061*"냉동"')
(1, '0.054*"판매" + 0.054*"포장" + 0.039*"봉지" + 0.033*"현재"')
(2, '0.145*"아이스팩" + 0.112*"추가" + 0.059*"구매" + 0.053*"옵션"')
(3, '0.166*"쇼핑" + 0.096*"동봉" + 0.035*"부분" + 0.034*"양해"')
(4, '0.266*"포장" + 0.163*"선물" + 0.041*"세트" + 0.037*"개별"')
(5, '0.329*"박스" + 0.111*"포장" + 0.059*"배송" + 0.037*"택배"')
(6, '0.149*"상품" + 0.084*"출고" + 0.049*"확인" + 0.043*"배송"')
(7, '0.150*"발송" + 0.070*"포장" + 0.032*"상품" + 0.032*"오늘"')
(8, '0.237*"제품" + 0.059*"포장" + 0.033*"참고" + 0.022*"배송"')
(9, '0.141*"아이스박스" + 0.100*"포장" + 0.065*"주문" + 0.049*"아이스팩"')


In [ ]:
pyLDAvis.enable_notebook()
vis4 = pyLDAvis.gensim_models.prepare(ldamodel4, corpus4, dictionary4)
pyLDAvis.display(vis4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 결제 인텐트 토픽 모델링

In [ ]:
df5 = data[(data['인텐트 (대)']=='결제')]
text5 = df5[['발화문']]
text5.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
15672,영수증이 필요해서 그런데요혹시 구매하고요청 드리면 제품 배송 시 영수증동봉해주실 수...
15673,택배와 함께 별도로 제공해 드리는 영수증은 없습니다. 영수증은 주문 내역에서 출력하...
15674,혹식 따로 영수증 첨부 가능할까요?
15675,간이 영수증 동봉하여 출고도와 드리겠습니다. 오늘도 즐거운 하루 되세요.
15676,현금영수증 발급 1건으로 묶어서 발급해주시면 안 되나요?


In [ ]:
text5['발화문'] = text5.apply(tokenize_korean_text, axis=1)
text5['발화문'] = text5['발화문'].apply(remove_stopwords)
tokenized_doc5 = text5['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc5[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-46-615ce43b5aab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text5['발화문'] = text5.apply(tokenize_korean_text, axis=1)


15672    [영수증, 구매, 요청, 제품, 배송, 영수증, 동봉]
15673    [택배, 별도, 제공, 영수증, 영수증, 주문, 출력]
15674                         [영수증, 첨부]
15675         [간이, 영수증, 동봉, 출고, 오늘, 하루]
15676                   [현금영수증, 발급, 발급]
Name: 발화문, dtype: object


<ipython-input-46-615ce43b5aab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text5['발화문'] = text5['발화문'].apply(remove_stopwords)


In [ ]:
dictionary5 = corpora.Dictionary(tokenized_doc5)
corpus5 = [dictionary5.doc2bow(text5) for text5 in tokenized_doc5]
print(corpus5[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(3, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_TOPICS = 10 # 10개의 토픽, k=10
ldamodel5 = gensim.models.ldamodel.LdaModel(corpus5, num_topics = NUM_TOPICS, id2word=dictionary5, passes=15)
topics5 = ldamodel5.print_topics(num_words=4)
for topic in topics5:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.277*"구매" + 0.060*"추가" + 0.031*"옵션" + 0.029*"구입"')
(1, '0.340*"주문" + 0.070*"금요일" + 0.067*"토요일" + 0.051*"배송"')
(2, '0.064*"입금" + 0.034*"처리" + 0.029*"환불" + 0.027*"추가"')
(3, '0.068*"상품" + 0.057*"현재" + 0.031*"양해" + 0.031*"예정"')
(4, '0.091*"문의" + 0.052*"택배" + 0.033*"배송" + 0.030*"매장"')
(5, '0.090*"쿠폰" + 0.076*"할인" + 0.052*"결제" + 0.051*"적용"')
(6, '0.115*"발송" + 0.082*"주문" + 0.055*"출고" + 0.050*"수령"')
(7, '0.298*"취소" + 0.060*"처리" + 0.036*"배송" + 0.029*"완료"')
(8, '0.153*"주문" + 0.036*"품절" + 0.034*"배송" + 0.032*"선물"')
(9, '0.136*"확인" + 0.075*"판매" + 0.037*"박스" + 0.032*"현재"')


In [ ]:
pyLDAvis.enable_notebook()
vis5 = pyLDAvis.gensim_models.prepare(ldamodel5, corpus5, dictionary5)
pyLDAvis.display(vis5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 교환|반품|환불 인텐트 토픽 모델링

In [ ]:
df6 = data[(data['인텐트 (대)']=='교환|반품|환불')]
text6 = df6[['발화문']]
text6.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
47179,지난주에 중량 미달로 문의 글 널어서 이번 주 내에 환불해준다고 하셨는데 언제 처리...
47180,답변이 늦어진 점 양해 부탁드립니다 최대한 빠른 처리 진행토록 하겠습니다.
47181,물품 오배송으로 교환받기로 했는데 물품은 수거해갔는데 교환 물품은 언제 오는 거죠?
47182,제품 일시 품절로 인해 금일 발송되십니다. 내 일 받아보시게 되시고요죄송한 마음을 ...
47183,배송 어렵다고 문자 받았었는데 환불 언제 되나요?


In [ ]:
text6['발화문'] = text6.apply(tokenize_korean_text, axis=1)
text6['발화문'] = text6['발화문'].apply(remove_stopwords)
tokenized_doc6 = text6['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc6[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-51-e66a0cc1c598>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text6['발화문'] = text6.apply(tokenize_korean_text, axis=1)


47179           [지난주, 중량, 미달, 문의, 환불, 처리]
47180               [답변, 양해, 최대한, 처리, 진행]
47181        [물품, 배송, 교환, 물품, 수거, 교환, 물품]
47182    [제품, 일시, 품절, 금일, 발송, 보시, 마음, 사은]
47183                        [배송, 문자, 환불]
Name: 발화문, dtype: object


<ipython-input-51-e66a0cc1c598>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text6['발화문'] = text6['발화문'].apply(remove_stopwords)


In [ ]:
dictionary6 = corpora.Dictionary(tokenized_doc6)
corpus6 = [dictionary6.doc2bow(text6) for text6 in tokenized_doc6]
print(corpus6[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(4, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_TOPICS = 10 # 10개의 토픽, k=10
ldamodel6 = gensim.models.ldamodel.LdaModel(corpus6, num_topics = NUM_TOPICS, id2word=dictionary6, passes=15)
topics6 = ldamodel6.print_topics(num_words=4)
for topic in topics6:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.153*"반품" + 0.143*"신청" + 0.058*"완료" + 0.043*"물건"')
(1, '0.308*"교환" + 0.070*"사진" + 0.042*"제품" + 0.039*"잘못"')
(2, '0.082*"제품" + 0.059*"박스" + 0.038*"폐기" + 0.037*"파손"')
(3, '0.115*"하나요" + 0.074*"반품" + 0.070*"신청" + 0.068*"택배"')
(4, '0.073*"확인" + 0.065*"접수" + 0.058*"진행" + 0.057*"회수"')
(5, '0.096*"문의" + 0.048*"확인" + 0.044*"상태" + 0.044*"주문"')
(6, '0.354*"환불" + 0.136*"처리" + 0.025*"구매" + 0.025*"입금"')
(7, '0.304*"반품" + 0.036*"판매" + 0.030*"처리" + 0.026*"택배"')
(8, '0.165*"취소" + 0.124*"배송" + 0.059*"주문" + 0.040*"처리"')
(9, '0.186*"상품" + 0.063*"교환" + 0.054*"발송" + 0.053*"배송"')


In [ ]:
pyLDAvis.enable_notebook()
vis6 = pyLDAvis.gensim_models.prepare(ldamodel6, corpus6, dictionary6)
pyLDAvis.display(vis6)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 전체 발화문에 대한 토픽 모델링

In [ ]:
t = data[['발화문']]
t.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,발화문
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...
1,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.
2,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...
3,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.
4,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...


In [ ]:
t['발화문'] = t.apply(tokenize_korean_text, axis=1)
t['발화문'] = t['발화문'].apply(remove_stopwords)
t_doc = t['발화문'].apply(lambda x: [word for word in x if len(word) > 1])
print(t_doc[:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-56-fb987e10d382>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['발화문'] = t.apply(tokenize_korean_text, axis=1)
<ipython-input-56-fb987e10d382>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

0                [컵밥, 사면, 무료, 배송, 죽도, 추가, 주문, 추가, 배송비]
1                                 [박스, 합포, 수도, 답변, 양해]
2    [해마, 기준, 볶음밥, 기준, 해마, 케이준, 볶음밥, 구매, 배송비, 내야, 하나요]
3                   [제품, 최소, 수량, 부과, 이상은, 시스템, 자동, 부과]
4                           [마트, 원데이, 무료, 배송, 결제, 배송비]
Name: 발화문, dtype: object


In [ ]:
diction = corpora.Dictionary(t_doc)
cor = [diction.doc2bow(t) for t in t_doc]
print(cor[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [ ]:
NUM_TOPICS = 20 # 20개의 토픽, k=20
lda = gensim.models.ldamodel.LdaModel(cor, num_topics = NUM_TOPICS, id2word=diction, passes=15)
topics = lda.print_topics(num_words=4)
for topic in topics:
    print(topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.297*"상품" + 0.189*"출고" + 0.067*"월요일" + 0.060*"예정"')
(1, '0.155*"경우" + 0.154*"판매" + 0.077*"옵션" + 0.071*"사은"')
(2, '0.166*"처리" + 0.105*"반품" + 0.089*"유통" + 0.084*"기한"')
(3, '0.135*"변경" + 0.104*"해당" + 0.088*"양해" + 0.085*"진행"')
(4, '0.160*"내일" + 0.121*"금요일" + 0.087*"출고" + 0.083*"문건"')
(5, '0.112*"차이" + 0.079*"문제" + 0.064*"기준" + 0.050*"기재"')
(6, '0.329*"확인" + 0.123*"고객" + 0.067*"다시" + 0.047*"번호"')
(7, '0.662*"배송" + 0.065*"묶음" + 0.063*"보관" + 0.052*"냉동"')
(8, '0.526*"주문" + 0.225*"오늘" + 0.113*"수령" + 0.036*"제주도"')
(9, '0.235*"현재" + 0.148*"도착" + 0.102*"하루" + 0.087*"지연"')
(10, '0.274*"가격" + 0.264*"포장" + 0.045*"금액" + 0.040*"생산"')
(11, '0.435*"제품" + 0.107*"지금" + 0.054*"냉장" + 0.045*"방금"')
(12, '0.276*"구매" + 0.113*"참고" + 0.073*"행사" + 0.068*"추가"')
(13, '0.482*"발송" + 0.291*"택배" + 0.052*"오전" + 0.041*"신청"')
(14, '0.281*"문의" + 0.078*"상품" + 0.072*"답변" + 0.049*"추석"')
(15, '0.259*"박스" + 0.104*"선물" + 0.084*"아이스팩" + 0.077*"아이스박스"')
(16, '0.115*"하나요" + 0.105*"캡슐" + 0.085*"지역" + 0.076*"이용"')
(17, '0.235*"토요

In [ ]:
pyLDAvis.enable_notebook()
avis = pyLDAvis.gensim_models.prepare(lda, cor, diction)
pyLDAvis.display(avis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
